# Kaggle
## Competition NFL Big Data Bowl

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
# Carregando os pacotes
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Statistic lib
from scipy import stats
from scipy.stats import skew, norm

# Sklearn lib
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder

# Models
from xgboost import XGBRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from mlxtend.regressor import StackingCVRegressor
from lightgbm import LGBMRegressor
import lightgbm as lgb
import xgboost as XGB
from sklearn.cluster import KMeans
from catboost import CatBoostRegressor


# Misc lib
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from functools import partial
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from IPython.display import Image
from tqdm import tqdm_notebook as tqdm

# Utils
import pandasql as ps
import re 
import math, string, os
import datetime
import tqdm

# Options
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_seq_items = 8000
pd.options.display.max_rows = 8000
pd.set_option('display.max_columns', None)
import gc
gc.enable()

In [ ]:
# Carregando os dados de treino
train = pd.read_csv('../data/train.csv', low_memory=False)

# Dataset criado gerando valores estatisticos
estat = pd.read_csv('../data/data.csv', low_memory=False)

# Merge com dataset estatistico (contem estatistica basica e avançada das features numericas)
train = pd.merge(train, estat, on='PlayId', how='outer')

#train = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2020/train.csv', low_memory=False)
print ("Data is ready !!")

In [ ]:
train.head()

# Criando as funções auxiliares de limpeza e conversao

In [ ]:
# Funcao para tratar os dados missing de cada variavel
def fill_na(data):
    data['WindDirection'].fillna('unknown',inplace=True)
    data['OffenseFormation'].fillna('unknown',inplace=True)
    data['StadiumType'].fillna('unknown',inplace=True)
    data['GameWeather'].fillna('unknown',inplace=True)
    data['FieldPosition'].fillna('NA',inplace=True)
    
    data['Temperature'].fillna(data['Temperature'].mean(), inplace=True)
    data['Humidity'].fillna(data['Humidity'].mean(), inplace=True)
    data['DefendersInTheBox'].fillna(math.ceil(data['DefendersInTheBox'].mean()),inplace=True)
    
# Funcao para agrupar as descricoes dos tipos de estadio
def agrupar_tipo_estadio(StadiumType):
    outdoor       = ['Outdoor', 'Outdoors', 'Cloudy', 'Heinz Field', 'Outdor', 'Ourdoor', 'Outside', 'Outddors', 'Outdoor Retr Roof-Open', 'Oudoor', 'Bowl']
    indoor_closed = ['Indoors', 'Indoor', 'Indoor, Roof Closed', 'Indoor, Roof Closed', 'Retractable Roof', 'Retr. Roof-Closed', 'Retr. Roof - Closed', 'Retr. Roof Closed']
    indoor_open   = ['Indoor, Open Roof', 'Open', 'Retr. Roof-Open', 'Retr. Roof - Open']
    dome_closed   = ['Dome', 'Domed, closed', 'Closed Dome', 'Domed', 'Dome, closed']
    dome_open     = ['Domed, Open', 'Domed, open']
    
    if StadiumType in outdoor:
        return 'outdoor'
    elif StadiumType in indoor_closed:
        return 'indoor_closed'
    elif StadiumType in indoor_open:
        return 'indoor_open'
    elif StadiumType in dome_closed:
        return 'dome_closed'
    elif StadiumType in dome_open:
        return 'dome_open'
    else:
        return 'unknown' # se for n/a
    
# Funcao para agrupar as descricoes dos estadios
def agrupar_estadio(Stadium):

    if Stadium == 'Broncos Stadium at Mile High':
        return 'Broncos Stadium At Mile High'
    
    elif Stadium in ('CenturyField', 'CenturyLink'):
        return 'CenturyLink Field'
    
    elif Stadium == 'EverBank Field':
        return 'Everbank Field'
    
    elif Stadium in ('FirstEnergy', 'FirstEnergy Stadium', 'FirstEnergyStadium'):
        return 'First Energy Stadium'
   
    elif Stadium == 'Lambeau field':
        return 'Lambeau Field'

    elif Stadium == 'Los Angeles Memorial Coliesum':
        return 'Los Angeles Memorial Coliseum'
    
    elif Stadium in ('M & T Bank Stadium', 'M&T Stadium'):
        return 'M&T Bank Stadium'

    elif Stadium in ('Mercedes-Benz Dome', 'Mercedes-Benz Superdome'):
        return 'Mercedes-Benz SuperDome'
    
    elif Stadium in ('MetLife Stadium', 'Metlife Stadium', 'MetLife'):
        return 'MetLife Stadium' 
    
    elif Stadium == 'NRG':
        return 'NRG Stadium' 

    elif Stadium == 'Oakland-Alameda County Coliseum':
        return 'Oakland Alameda-County Coliseum' 
    
    elif Stadium == 'Paul Brown Stdium':
        return 'Paul Brown Stadium' 

    elif Stadium == 'Twickenham':
        return 'Twickenham Stadium' 
    
    else:
        return Stadium
    
# Funcao para agrupar a localizacao do estadio e do jogo
def agrupar_local(Location):

    if Location == "Arlington, Texas":
        return "Arlington, TX"
    elif Location in ("Baltimore, Maryland","Baltimore, Md."):
        return "Baltimore, MD"
    elif Location == "Charlotte, North Carolina":
        return "Charlotte, NC"
    elif Location == "Chicago. IL":
        return "Chicago, IL"
    elif Location == "Cincinnati, Ohio":
        return "Cincinnati, OH"
    elif Location in ("Cleveland","Cleveland Ohio","Cleveland, Ohio","Cleveland,Ohio"):
        return "Cleveland, OH"
    elif Location == "Detroit":
        return "Detroit, MI"
    elif Location == "E. Rutherford, NJ" or Location == "East Rutherford, N.J.":
        return "East Rutherford, NJ"
    elif Location == "Foxborough, Ma":
        return "Foxborough, MA"
    elif Location == "Houston, Texas":
        return "Houston, TX"
    elif Location in ("Jacksonville Florida","Jacksonville, Fl","Jacksonville, Florida"):
        return "Jacksonville, FL"
    elif Location == "London":
        return "London, England"
    elif Location == "Los Angeles, Calif.":
        return "Los Angeles, CA"
    elif Location == "Miami Gardens, Fla.":
        return "Miami Gardens, FLA"
    elif Location in ("New Orleans","New Orleans, La."):
        return "New Orleans, LA"
    elif Location == "Orchard Park NY":
        return "Orchard Park, NY"
    elif Location == "Philadelphia, Pa.":
        return "Philadelphia, PA"
    elif Location == "Pittsburgh":
        return "Pittsburgh, PA"
    elif Location == "Seattle":
        return "Seattle, WA"
    else:
        return Location
    
# Funcao para agrupar o gramado do estadio
def agrupar_gramado(Turf):
    if Turf == 'Artifical':
        return 'Artificial'
    
    elif Turf in ('FieldTurf', 'Field turf'):
        return 'Field Turf'

    elif Turf in ('FieldTurf360', 'FieldTurf 360'):
        return 'Field Turf 360'

    elif Turf in ('Natural', 'Natural grass', 'Naturall Grass', 'grass', 'natural grass', 'SISGrass', 'Natural Grass'):
        return "Grass"

    elif Turf == "UBU Sports Speed S5-M":
        return "UBU Speed Series-S5-M"

    else:
        return Turf

# Funcao para agrupar os dados de direcao do vento
def agrupa_wind_direction(WindDirection):
    wd = str(WindDirection).upper()
    
    if wd == 'N' or 'FROM N' in wd:
        return 'north'
    if wd == 'S' or 'FROM S' in wd:
        return 'south'
    if wd == 'W' or 'FROM W' in wd:
        return 'west'
    if wd == 'E' or 'FROM E' in wd:
        return 'east'
    
    if 'FROM SW' in wd or 'FROM SSW' in wd or 'FROM WSW' in wd:
        return 'south west'
    if 'FROM SE' in wd or 'FROM SSE' in wd or 'FROM ESE' in wd:
        return 'south east'
    if 'FROM NW' in wd or 'FROM NNW' in wd or 'FROM WNW' in wd:
        return 'north west'
    if 'FROM NE' in wd or 'FROM NNE' in wd or 'FROM ENE' in wd:
        return 'north east'
    
    if 'NW' in wd or 'NORTHWEST' in wd:
        return 'north west'
    if 'NE' in wd or 'NORTH EAST' in wd:
        return 'north east'
    if 'SW' in wd or 'SOUTHWEST' in wd:
        return 'south west'
    if 'SE' in wd or 'SOUTHEAST' in wd:
        return 'south east'

    return 'unknown'

# Funcao para agrupar as descricoes de clima
def agrupar_clima(GameWeather):
    chuva   = ['Rainy', 'Rain Chance 40%', 'Showers',
               'Cloudy with periods of rain, thunder possible. Winds shifting to WNW, 10-20 mph.',
               'Scattered Showers', 'Cloudy, Rain', 'Rain shower', 'Light Rain', 'Rain']
    nublado = ['Cloudy, light snow accumulating 1-3"', 'Party Cloudy', 'Cloudy, chance of rain',
               'Coudy', 'Cloudy, 50% change of rain', 'Rain likely, temps in low 40s.',
               'Cloudy and cold', 'Cloudy, fog started developing in 2nd quarter',
               'Partly Clouidy', '30% Chance of Rain', 'Mostly Coudy', 'Cloudy and Cool',
               'cloudy', 'Partly cloudy', 'Overcast', 'Hazy', 'Mostly cloudy', 'Mostly Cloudy',
               'Partly Cloudy', 'Cloudy']
    limpo   = ['Partly clear', 'Sunny and clear', 'Sun & clouds', 'Clear and Sunny',
               'Sunny and cold', 'Sunny Skies', 'Clear and Cool', 'Clear and sunny',
               'Sunny, highs to upper 80s', 'Mostly Sunny Skies', 'Cold',
               'Clear and warm', 'Sunny and warm', 'Clear and cold', 'Mostly sunny',
               'T: 51; H: 55; W: NW 10 mph', 'Clear Skies', 'Clear skies', 'Partly sunny',
               'Fair', 'Partly Sunny', 'Mostly Sunny', 'Clear', 'Sunny']
    neve    = ['Heavy lake effect snow', 'Snow']
    none    = ['N/A Indoor', 'Indoors', 'Indoor', 'N/A (Indoors)', 'Controlled Climate']

    
    if GameWeather in chuva:
        return 'chuva'
    elif GameWeather in nublado:
        return 'nublado'
    elif GameWeather in limpo:
        return 'limpo'
    elif GameWeather in neve:
        return 'neve'
    elif GameWeather in none:
        return 'none'
    else:
        return 'none' # se for n/a
    
# Funcao para converter a velocidade do vento
def convert_wind_speed(WindSpeed):
    ws = str(WindSpeed)

    if ws.isdigit():
        return int(ws)

    if '-' in ws:
        return int(ws.split('-')[0])

    if ws.split(' ')[0].isdigit():
        return int(ws.split(' ')[0])

    if 'mph' in ws.lower():
        return int(ws.lower().split('mph')[0])
    else:
        return 0
    
    
# Funcao para converter altura de feet-inches para centimetros
def convert_to_cm(ft_in):
    h_ft   = int(ft_in.split('-')[0])
    h_inch = int(ft_in.split('-')[1])
    h_inch += h_ft * 12
    h_cm = round(h_inch * 2.54, 1)
    #print("Your height is : %d cm." % h_cm)   
    
    return h_cm

# Funcao para converter peso em lbs para kg
def convert_to_kg(lbs):
    kg = lbs * 0.45359237
    #print("The weight is", kg, "in kilograms")
    
    return kg

# Funcao para converter temperatura Fahrenheit para Celsius
def convert_to_celsius(fah):
    celsius = (fah - 32) * 5.0/9.0
    #print("Temperature:", fah, "Fahrenheit = ", celsius, " C")
    return celsius

# Funcao para converter as features de data e extrair dia, mes, ano, hora, minuto, segundo
def convert_data(data):
    data['PlayerBirthDate'] = data['PlayerBirthDate'].apply(lambda x: datetime.datetime.strptime(x, "%m/%d/%Y"))
    data['PlayerBirthDate_day'] = data['PlayerBirthDate'].dt.day.astype(int)
    data['PlayerBirthDate_month'] = data['PlayerBirthDate'].dt.month.astype(int)
    data['PlayerBirthDate_year'] = data['PlayerBirthDate'].dt.year.astype(int)

    data['TimeSnap'] = data['TimeSnap'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ"))
    data['TimeSnap_min'] = data['TimeSnap'].dt.minute.astype(int)
    data['TimeSnap_seg'] = data['TimeSnap'].dt.second.astype(int)
    
    data['TimeHandoff'] = data['TimeHandoff'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ"))
    data['TimeHandoff_min'] = data['TimeHandoff'].dt.minute.astype(int)
    data['TimeHandoff_seg'] = data['TimeHandoff'].dt.second.astype(int)
    
    
# Funcao para converter uma string horario em segundos
def str_to_seconds(time):
    time = time.split(':')
    sec = int(time[0])*60 + int(time[1]) + int(time[2])/60
    return sec

# Funcao para a metrica de validacao do modelo
def funcao_crps(labels,predictions) :
    y_pred = np.zeros((len(labels),199))
    y_ans = np.zeros((len(labels),199))
    j = np.array(range(199))
    for i,(p,t) in enumerate(zip(np.round(scaler.inverse_transform(predictions)),labels)) :
        k2 = j[j>=p-10]
        y_pred[i][k2]=(k2+10-p)*0.05
        k1 = j[j>=p+10]
        y_pred[i][k1]= 1.0
        k3 = j[j>=t]
        y_ans[i][k3]= 1.0
                           
    return 'CRPS: ', np.sum((y_pred-y_ans)**2)/(199*y_pred.shape[0]), False

# Feature Engineering

In [ ]:
# Funcao para realizar feature engineering no dataset (treino ou teste)
def feature_engineering(df): 
    
    # Limpeza e conversao dos dados
    fill_na(df)
    convert_data(df)
    
    # Conversao de algumas features
    df['PlayerHeight']  = df['PlayerHeight'].apply(convert_to_cm)
    df['PlayerWeight']  = df['PlayerWeight'].apply(convert_to_kg)
    df['Temperature']   = df['Temperature'].apply(convert_to_celsius)
    df['StadiumType']   = df['StadiumType'].apply(agrupar_tipo_estadio)
    df['Stadium']       = df['Stadium'].apply(agrupar_estadio)
    df['Location']      = df['Location'].apply(agrupar_local)
    df['Turf']          = df['Turf'].apply(agrupar_gramado)
    df['WindDirection'] = df['WindDirection'].apply(agrupa_wind_direction)
    df['WindSpeed']     = df['WindSpeed'].apply(convert_wind_speed)
    df['GameWeather']   = df['GameWeather'].apply(agrupar_clima)


    # Corrigindo a feature Stadium
    df.loc[df['Stadium'] == 'MetLife Stadium', 'StadiumType'] = 'outdoor'
    df.loc[df['Stadium'] == 'StubHub Center', 'StadiumType'] = 'outdoor'    
    
    # Nova feature com a diferença entre o tempo de lançamento da bola até quando o jogador captura
    df['TimeDifer'] = df.apply(lambda row: (row['TimeHandoff'] - row['TimeSnap']).total_seconds(), axis=1)            
         
    # Nova feature para indicar se é o jogador que esta realizando a jogada (corredor)
    df['IsRusher'] = df['NflId'] == df['NflIdRusher']
    
    # Novas features com base no horario do jogo
    df['Morning']   = df['GameClock'].apply(lambda x : 1 if (int(x[0:2]) >=0 and int(x[0:2]) <12) else 0)
    df['Afternoon'] = df['GameClock'].apply(lambda x : 1 if (int(x[0:2]) <18 and int(x[0:2]) >=12) else 0)
    df['Evening']   = df['GameClock'].apply(lambda x : 1 if (int(x[0:2]) >= 18 and int(x[0:2]) < 24) else 0)
    df['GameClock'] = df['GameClock'].apply(str_to_seconds) 
    
    # Criando novas features com dados de Distance, YardLine e DefendersInTheBox
    df['seconds_need_to_first_down'] = (df['Distance']*0.9144)/df['Dis']
    df['seconds_need_to_YardsLine'] = (df['YardLine']*0.9144)/df['Dis']    
    df['DefendersInTheBox_vs_Distance'] = df['DefendersInTheBox'] / df['Distance']
    
    # Ordenacao do dataset e renovando o index
    df = df.sort_values(by=['PlayId', 'Team', 'IsRusher']).reset_index()
    
    # Removendo colunas que não serão utilizadas
    df = df.drop(['index','Yards','GameId','PlayId','TimeHandoff','TimeSnap','PlayerBirthDate'], axis=1)

    # Atribuindo media para os demais dados missing
    #df_median = df.median()
    #df.fillna(df_median, inplace=True)

    # Removendo todas as variaveis categoricas
    #cat_features = []
    #for col in df.columns:
    #    if df[col].dtype =='object':
    #        cat_features.append(col)

    #df = df.drop(cat_features, axis=1)
    
    return df

In [ ]:
# Criando um novo dataset aplicando Feature Engineering
train_df = feature_engineering(train)

In [ ]:
# Executar somente para alguns modelos que nao tratam valores INF ou NAN
train_df.replace(-np.inf,0,inplace=True)
train_df.replace(np.inf,0,inplace=True)
train_df.fillna(0, inplace=True)

In [ ]:
train_df.head()

In [ ]:
train_df.shape

In [ ]:
# Transformando algumas variaveis categoricas para inteiro
cleanup_nums = {"StadiumType": {"outdoor"      : 73, 
                                "indoor_closed": 17,
                                "dome_closed"  : 5, 
                                "indoor_open"  : 2, 
                                "dome_open"    : 0,
                                "unknown"      : 0
                               },
                "Turf":        {"A-Turf Titan"           : 33, 
                                "Artificial"             : 94,
                                "DD GrassMaster"         : 15, 
                                "Field Turf"             : 16,
                                "Field Turf 360"         : 2, 
                                "Grass"                  : 56,
                                "Twenty-Four/Seven Turf" : 1,
                                "UBU Speed Series-S5-M"  : 8
                               },
                "GameWeather": {"nublado" : 49, 
                                "limpo"   : 41,
                                "none"    : 20, 
                                "chuva"   : 10,
                                "neve"    : 0
                               },
                "WindDirection": {"unknown"    : 30, 
                                  "south west" : 15,
                                  "north west" : 13, 
                                  "north east" : 13,
                                  "south east" : 10,
                                  "south"      : 6,
                                  "north"      : 5,
                                  "west"       : 5,
                                  "east"       : 3
                                 },               
               
               }
train_df.replace(cleanup_nums, inplace=True)
train_df.head()

In [ ]:
cat_features_names = ['Team','NflId','DisplayName','JerseyNumber','NflIdRusher','PlayerCollegeName',
                      'Stadium','Location','PlayDirection','DisplayName','StadiumType',
                      'Turf','GameWeather','WindDirection','Position','Season','HomeTeamAbbr','VisitorTeamAbbr',
                      'PossessionTeam','FieldPosition','Quarter','OffenseFormation','DefensePersonnel',
                      'OffensePersonnel']
cat_features = [train.columns.get_loc(col) for col in cat_features_names]
print(cat_features)

In [ ]:
train_df.head()

In [ ]:
train_df_new = train_df.copy()

In [ ]:
# Removendo todas as variaveis categoricas
drop_features = []
for col in train_df_new.columns:
    if train_df_new[col].dtype =='object':
        drop_features.append(col)

train_df_new = train_df_new.drop(drop_features, axis=1)

In [ ]:
# Executar somente para alguns modelos que nao tratam valores INF ou NAN
train_df_new.replace(-np.inf,0,inplace=True)
train_df_new.replace(np.inf,0,inplace=True)
train_df_new.fillna(0, inplace=True)

In [ ]:
train_df_new = train_df_new.drop(['NflId','NflIdRusher'], axis=1)

In [ ]:
train_df_new.head()

In [ ]:
train_df_new["JerseyNumber"] = train_df_new["JerseyNumber"].astype(int)
train_df_new["Season"] = train_df_new["Season"].astype(int)
train_df_new["Quarter"] = train_df_new["Quarter"].astype(int)
train_df_new["Week"] = train_df_new["Week"].astype(int)
train_df_new["StadiumType"] = train_df_new["StadiumType"].astype(int)
train_df_new["Turf"] = train_df_new["Turf"].astype(int)
train_df_new["GameWeather"] = train_df_new["GameWeather"].astype(int)
train_df_new["WindDirection"] = train_df_new["WindDirection"].astype(int)
train_df_new["Morning"] = train_df_new["Morning"].astype(int)
train_df_new["Afternoon"] = train_df_new["Afternoon"].astype(int)
train_df_new["Evening"] = train_df_new["Evening"].astype(int)
train_df_new["PlayerBirthDate_day"] = train_df_new["PlayerBirthDate_day"].astype(int)
train_df_new["PlayerBirthDate_month"] = train_df_new["PlayerBirthDate_month"].astype(int)
train_df_new["PlayerBirthDate_year"] = train_df_new["PlayerBirthDate_year"].astype(int)

# Criação e Validação dos Modelos de ML

In [ ]:
# Fazendo uma limpeza na memoria
gc.collect()

In [ ]:
# Setup cross validation folds
kf = 5
folds = KFold(n_splits=kf, shuffle=False, random_state=42)
print(str(kf) + ' Folds para treino...')

In [ ]:
# Identificando as features para o modelo
features = list(train_df_new.columns)

In [ ]:
# Criando matriz de treino
train_data = np.zeros((509762//22,len(features))) 

for i in tqdm.tqdm(range(0,509762,22)):
    count=0
    for c in features:
        train_data[i//22][count] = train_df_new[c][i]
        count+=1

In [ ]:
train_data.shape

In [ ]:
# Split de dados e label
X_train = pd.DataFrame(data=train_data,columns=features)
y_train_ = np.array([train["Yards"][i] for i in range(0,509762,22)])

In [ ]:
# Normalizacao das features 
y_tr = np.zeros(len(y_train_),dtype=np.float)
for i in range(len(y_tr)):
    y_tr[i]=(y_train_[i])

scaler = preprocessing.StandardScaler()
scaler.fit([[y] for y in y_tr])
y_tr = np.array([y[0] for y in scaler.transform([[y] for y in y_tr])])

In [ ]:
feature_importance_df = pd.DataFrame()
oof = np.zeros(len(X_train))
tr_rmse  = []
val_rmse = []
models   = []

for fold_, (tr_idx, val_idx) in enumerate(folds.split(X_train,y_tr)):

    tr_x, vl_x = X_train.iloc[tr_idx][features], X_train.iloc[val_idx][features]
    tr_y, vl_y = y_tr[tr_idx], y_tr[val_idx]

    print({'Fold: ':fold_, 'Train size':len(tr_x), 'Val size':len(vl_x)})

    tr_data = lgb.Dataset(tr_x, label=tr_y)
    vl_data = lgb.Dataset(vl_x, label=vl_y)  
    
    """model = lgb.LGBMRegressor(n_estimators=1000,
                              learning_rate=0.01,
                              boosting="gbdt",
                              feature_fraction=0.9,
                              subsample=0.2,
                              subsample_freq=1,
                              num_leaves=20,
                              metric='rmse')
    model.fit(tr_x, tr_y,
              eval_set=[(vl_x, vl_y)],
              eval_metric=funcao_crps,
              early_stopping_rounds=100,
              verbose=200)"""
    
    """model = XGB.XGBRegressor(objective='reg:squarederror',
                             max_depth=8,
                             n_estimators=100,
                             min_child_weight=300,
                             colsample_bytree=0.8,
                             subsample=0.8,
                             eta=0.01,
                             metric='rmse',
                             seed=42)

    model.fit(tr_x, tr_y,
              #eval_metric=funcao_crps,
              eval_set=[(vl_x, vl_y)],
              verbose=200,
              early_stopping_rounds = 100)"""
    
    
    model = CatBoostRegressor(n_estimators=1000,
                              learning_rate=0.01)

    model.fit(tr_x, tr_y,
              eval_set=[(vl_x, vl_y)],
              early_stopping_rounds=100,
              use_best_model=True,
              verbose=20)

    oof[val_idx] = model.predict(vl_x)
    
    val_score = mean_squared_error(vl_y, oof[val_idx])
    val_rmse.append(val_score)
    
    tr_score = mean_squared_error(tr_y, model.predict(tr_x))
    tr_rmse.append(tr_score)
    
    models.append(model)
    
    # Feature importance
    fold_importance_df = pd.DataFrame()
    fold_importance_df["Feature"] = tr_x.columns
    fold_importance_df["importance"] = model.feature_importances_[:len(tr_x.columns)]
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
gc.collect()

In [ ]:
_ = XGB.plot_importance(models[3], importance_type='gain')

In [ ]:
cols = (feature_importance_df[["Feature", "importance"]]
        .groupby("Feature")
        .mean()
        .sort_values(by="importance", ascending=False)[:50].index)
best_features = feature_importance_df.loc[feature_importance_df.Feature.isin(cols)]

plt.figure(figsize=(18,18))
sns.barplot(x="importance", y="Feature", data=best_features.sort_values(by="importance",ascending=False))
plt.title('XBoost Features Importance (media dos folds)')
plt.tight_layout()

In [ ]:
_ = lgb.plot_importance(models[3], importance_type='gain')

In [ ]:
cols = (feature_importance_df[["Feature", "importance"]]
        .groupby("Feature")
        .mean()
        .sort_values(by="importance", ascending=False)[:50].index)
best_features = feature_importance_df.loc[feature_importance_df.Feature.isin(cols)]

plt.figure(figsize=(18,18))
sns.barplot(x="importance", y="Feature", data=best_features.sort_values(by="importance",ascending=False))
plt.title('LightGBM Features Importance (media dos folds)')
plt.tight_layout()

In [ ]:
# Validacao do modelo
# Imprimir o score de treino e teste

mean_rmse_tr = np.mean(tr_rmse)
std_rmse_tr =  np.std(tr_rmse)

mean_rmse_val =  np.mean(val_rmse)
std_rmse_val =  np.std(val_rmse)

all_rmse = mean_squared_error(oof,y_tr)

print("Score de Treino")
print("Média RMSE: %.5f, std: %.5f." % (mean_rmse_tr, std_rmse_tr),'\n')

print("Score de Validação")
print("Média RMSE: %.5f, std: %.5f." % (mean_rmse_val, std_rmse_val),'\n')

print("Geral: %.5f." % (all_rmse),'\n')

print("CRPS: %.6f." % (funcao_crps(y_tr,oof)[1]))

# Realizando a submissão

In [ ]:
# Funcao para realizar feature engineering no dataset de teste
def feature_engineering_test(df): 
    
    # Limpeza e conversao dos dados
    fill_na(df)
    convert_data(df)
    
    df_median = df.median()
    
    # Conversao de algumas features
    df['PlayerHeight']  = df['PlayerHeight'].apply(convert_to_cm)
    df['PlayerWeight']  = df['PlayerWeight'].apply(convert_to_kg)
    df['Temperature']   = df['Temperature'].apply(convert_to_celsius)
    df['StadiumType']   = df['StadiumType'].apply(agrupar_tipo_estadio)
    df['Stadium']       = df['Stadium'].apply(agrupar_estadio)
    df['Location']      = df['Location'].apply(agrupar_local)
    df['Turf']          = df['Turf'].apply(agrupar_gramado)
    df['WindDirection'] = df['WindDirection'].apply(agrupa_wind_direction)
    df['WindSpeed']     = df['WindSpeed'].apply(convert_wind_speed)
    df['GameWeather']   = df['GameWeather'].apply(agrupar_clima)


    # Corrigindo a feature Stadium
    df.loc[df['Stadium'] == 'MetLife Stadium', 'StadiumType'] = 'outdoor'
    df.loc[df['Stadium'] == 'StubHub Center', 'StadiumType'] = 'outdoor'    
    
    # Nova feature com a diferença entre o tempo de lançamento da bola até quando o jogador captura
    df['TimeDifer'] = df.apply(lambda row: (row['TimeHandoff'] - row['TimeSnap']).total_seconds(), axis=1)            
         
    # Nova feature para indicar se é o jogador que esta realizando a jogada (corredor)
    df['IsRusher'] = df['NflId'] == df['NflIdRusher']
    
    # Novas features com base no horario do jogo
    df['Morning']   = df['GameClock'].apply(lambda x : 1 if (int(x[0:2]) >=0 and int(x[0:2]) <12) else 0)
    df['Afternoon'] = df['GameClock'].apply(lambda x : 1 if (int(x[0:2]) <18 and int(x[0:2]) >=12) else 0)
    df['Evening']   = df['GameClock'].apply(lambda x : 1 if (int(x[0:2]) >= 18 and int(x[0:2]) < 24) else 0)
    df['GameClock'] = df['GameClock'].apply(str_to_seconds) 
    
    # Criando novas features com dados de Distance, YardLine e DefendersInTheBox
    df['seconds_need_to_first_down'] = (df['Distance']*0.9144)/df['Dis']
    df['seconds_need_to_YardsLine'] = (df['YardLine']*0.9144)/df['Dis']    
    df['DefendersInTheBox_vs_Distance'] = df['DefendersInTheBox'] / df['Distance']
    
    # Ordenacao do dataset e renovando o index
    df = df.sort_values(by=['PlayId', 'Team', 'IsRusher']).reset_index()
    
    # Removendo colunas que não serão utilizadas
    df = df.drop(['index', 'GameId','PlayId','NflId', 'DisplayName','NflIdRusher', 'TimeHandoff', 'TimeSnap', 'PlayerBirthDate'], axis=1)

    # Atribuindo media para os demais dados missing
    df.fillna(df_median, inplace=True)

    # Executar somente para alguns modelos que nao tratam valores INF ou NAN
    #train_df.replace(-np.inf,0,inplace=True)
    #train_df.replace(np.inf,0,inplace=True)

    # Removendo todas as variaveis categoricas
    cat_features = []
    for col in df.columns:
        if df[col].dtype =='object':
            cat_features.append(col)

    df = df.drop(cat_features, axis=1)
    
    return df

In [ ]:
from kaggle.competitions import nflrush

pd.options.mode.chained_assignment = None
index = 0

env = nflrush.make_env()

for (test, sample_prediction_df) in tqdm.tqdm(env.iter_test()):
    df_test = feature_engineering_test(test)
    
    count=0
    test_data = np.zeros((1,len(features)))

    for c in features:
        if c in df_test:
            try:
                test_data[0][count] = df_test[c][index]
            except:
                test_data[0][count] = np.nan
            count+=1
    
    y_pred = np.zeros(199)        
    y_pred_p = np.sum(np.round(scaler.inverse_transform([model.predict(test_data)[0] for model in models])))/n_folds
    y_pred_p += 99
    
    for j in range(199):
        if j>=y_pred_p+10:
            y_pred[j]=1.0
        elif j>=y_pred_p-10:
            y_pred[j]=(j+10-y_pred_p)*0.05
    
    env.predict(pd.DataFrame(data=[y_pred],columns=sample_prediction_df.columns))
    
    index += 22
    
env.write_submission_file()

print([filename for filename in os.listdir('/kaggle/working') if '.csv' in filename])